In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki
from FairRanking.models.BaseDirectRanker import convert_data_to_tensors
from FairRanking.models.DebiasClassifier import DebiasClassifier
from FairRanking.TrainingFunctions.DebiasClassifierTrain import train
from FairRanking.helpers import rND_torch, nDCG_cls_no_model, auc_estimator, group_pairwise_accuracy, auc_estimator2

In [15]:
data = Law('Race','/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Adult('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Compas('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Wiki()
#X_train, s_train, y_train, X_val, s_val, y_val, X_test, s_test, y_test = convert_data_to_tensors(data, build_pairs=False)
full_data = data.get_data()

In [3]:
#(X_train, s_train, y_train), (X_val, s_val, y_val), (X_test, s_test, y_test) = data.get_data()

In [16]:
model = DebiasClassifier(num_features=full_data[0][0].shape[1],
                    kernel_initializer=nn.init.normal_,
                    hidden_layers=[64, 32],
                    bias_layers=[64, 32, 16, 2],
                    num_relevance_classes=len(torch.unique(full_data[0][1]))
            )

In [17]:

model = train(model, full_data, schedule = [1,1], adv_lr = 0.001, main_lr=0.001, n_epochs = 1000, random_seed=42)

0\1000	 Loss: 2.3707756996154785	 Sens Loss: 0.5777342915534973
100\1000	 Loss: 2.093188762664795	 Sens Loss: 0.537274956703186
200\1000	 Loss: 1.6184735298156738	 Sens Loss: 0.5264160633087158
300\1000	 Loss: 1.2949799299240112	 Sens Loss: 0.5258516073226929
400\1000	 Loss: 1.064518928527832	 Sens Loss: 0.5434296727180481
500\1000	 Loss: 0.9292394518852234	 Sens Loss: 0.5352419018745422
600\1000	 Loss: 0.8246256709098816	 Sens Loss: 0.5456917881965637
700\1000	 Loss: 0.7095827460289001	 Sens Loss: 0.537748396396637
800\1000	 Loss: 0.5820266008377075	 Sens Loss: 0.5326143503189087
900\1000	 Loss: 0.4981110692024231	 Sens Loss: 0.5494025349617004


In [18]:
data_idx = 2
model.eval()
with torch.no_grad():
    #h = model.forward_extracted_features(X_test0)
    #print(h.shape)
    #print(torch.amax(h, dim=1))
    shuffled = full_data[data_idx][0][torch.randperm(full_data[data_idx][0].size(0))]
    pred = model.predict_proba(full_data[data_idx][0])
    #pred = model(full_data[data_idx][0], torch.zeros_like(full_data[data_idx][0]))
    _, sorted_pred_indices = torch.sort(pred, dim=0)
    _, sorted_y_indices = torch.sort(full_data[data_idx][1], dim=0)
    # get the counts of correct and falsy ordered for accuracy
    #_, counts = torch.unique(full_data[0][0][sorted_pred_indices] == full_data[0][0][sorted_y_indices], return_counts=True)
    #print(counts[1]/(counts[0]+counts[1]))
    print('GPA: ', group_pairwise_accuracy(pred, full_data[data_idx][1], full_data[data_idx][2]))
    pred = torch.argmax(pred, dim=1)
    print('nDCG: ', nDCG_cls_no_model(pred, full_data[data_idx][1], esti=False, trec=True))
    #auc_pred = model.predict_proba(full_data[data_idx][0])
    print('AUC: ', auc_estimator(pred, full_data[data_idx][1]))
    print('rND: ', rND_torch(pred, torch.argmax(full_data[data_idx][2], dim=1)))

GPA:  0.17280611932224488
nDCG:  0.0009775172220543027
AUC:  0.5905431209807439
rND:  0.16335662423035677


In [6]:
for name, p in model.named_parameters():
    print(name)

feature_layers.0.weight
feature_layers.0.bias
feature_layers.1.weight
feature_layers.1.bias
feature_layers.2.weight
feature_layers.2.bias
debias_layers.0.weight
debias_layers.0.bias
debias_layers.1.weight
debias_layers.1.bias
debias_layers.2.weight
debias_layers.2.bias
debias_layers.3.weight
debias_layers.3.bias
additional_main_layers.0.weight
additional_main_layers.0.bias
additional_main_layers.1.weight
additional_main_layers.1.bias
additional_main_layers.2.weight
additional_main_layers.2.bias
output_layer.weight
output_layer.bias
